In [4]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
@Author: Bruce Shuyue Jia
@Date: Jan 30, 2021
"""

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [5]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.5):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential([layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim), ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out = self.layernorm2(out1 + ffn_output)
        return out


class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = tf.reshape(x, [-1, maxlen, embed_dim])
        out = x + positions
        return out


In [18]:

# Read Training Data
train_data = pd.read_csv('Downloads/EEG-GAL-DATA/train/Signal/subj10_series1_data.csv',index_col = 'id')
train_data = np.array(train_data).astype('float32')

# Read Training Labels
train_labels = pd.read_csv('Downloads/EEG-GAL-DATA/train/Label/subj10_series1_events.csv',index_col = 'id')
train_labels = np.array(train_labels).astype('float32')
train_labels = np.squeeze(train_labels)

# Read Testing Data
test_data = pd.read_csv('Downloads/EEG-GAL-DATA/train/Test/Signal/subj4_series1_data.csv',index_col = 'id')
test_data = np.array(test_data).astype('float32')

# Read Testing Labels
test_labels = pd.read_csv('Downloads/EEG-GAL-DATA/train/Test/Label/subj4_series1_events.csv',index_col = 'id')
test_labels = np.array(test_labels).astype('float32')
test_labels = np.squeeze(test_labels)


class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.5):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential([layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim), ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out = self.layernorm2(out1 + ffn_output)
        return out


class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = tf.reshape(x, [-1, maxlen, embed_dim])
        out = x + positions
        return out


maxlen = 1     # Only consider 3 input time points
embed_dim = 32  # Features of each time point
num_heads = 8   # Number of attention heads
ff_dim = 64     # Hidden layer size in feed forward network inside transformer

# Input Time-series
#inputs = layers.Input(shape=(maxlen*embed_dim,))
inputs = layers.Input(shape=(maxlen*embed_dim,))
embedding_layer = TokenAndPositionEmbedding(maxlen, embed_dim)
x = embedding_layer(inputs)

# Encoder Architecture
transformer_block_1 = TransformerBlock(embed_dim=embed_dim, num_heads=num_heads, ff_dim=ff_dim)
transformer_block_2 = TransformerBlock(embed_dim=embed_dim, num_heads=num_heads, ff_dim=ff_dim)
x = transformer_block_1(x)
x = transformer_block_2(x)

# Output
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(6, activation="sigmoid")(x)



In [19]:
outputs.shape

TensorShape([None, 6])

In [20]:
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-4),
              loss="binary_crossentropy",
              metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Recall()])

history = model.fit(
    train_data, train_labels, batch_size=128, epochs=1000, validation_data=(test_data, test_labels)
)

Epoch 1/1000


/home/evo/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


2050/2050 [==============================] - 14s 6ms/step - loss: 0.1625 - precision_4: 0.0204 - binary_accuracy: 0.9604 - recall_4: 0.0220 - val_loss: 0.0979 - val_precision_4: 0.0000e+00 - val_binary_accuracy: 0.9793 - val_recall_4: 0.0000e+00
Epoch 2/1000
2050/2050 [==============================] - 12s 6ms/step - loss: 0.1071 - precision_4: 0.0402 - binary_accuracy: 0.9803 - recall_4: 6.2092e-04 - val_loss: 0.1091 - val_precision_4: 0.0000e+00 - val_binary_accuracy: 0.9793 - val_recall_4: 0.0000e+00
Epoch 3/1000
2050/2050 [==============================] - 12s 6ms/step - loss: 0.0979 - precision_4: 0.1000 - binary_accuracy: 0.9805 - recall_4: 9.8039e-05 - val_loss: 0.1150 - val_precision_4: 0.0000e+00 - val_binary_accuracy: 0.9793 - val_recall_4: 0.0000e+00
Epoch 4/1000
2050/2050 [==============================] - 12s 6ms/step - loss: 0.0941 - precision_4: 0.0000e+00 - binary_accuracy: 0.9806 - recall_4: 0.0000e+00 - val_loss: 0.1190 - val_precision_4: 0.0000e+00 - val_binary_accur

2050/2050 [==============================] - 12s 6ms/step - loss: 0.0610 - precision_4: 0.5249 - binary_accuracy: 0.9806 - recall_4: 0.0148 - val_loss: 0.2592 - val_precision_4: 0.0000e+00 - val_binary_accuracy: 0.9793 - val_recall_4: 0.0000e+00
Epoch 33/1000
2050/2050 [==============================] - 12s 6ms/step - loss: 0.0604 - precision_4: 0.5335 - binary_accuracy: 0.9806 - recall_4: 0.0182 - val_loss: 0.2608 - val_precision_4: 0.0000e+00 - val_binary_accuracy: 0.9793 - val_recall_4: 0.0000e+00
Epoch 34/1000
2050/2050 [==============================] - 12s 6ms/step - loss: 0.0599 - precision_4: 0.5480 - binary_accuracy: 0.9806 - recall_4: 0.0190 - val_loss: 0.2654 - val_precision_4: 0.0000e+00 - val_binary_accuracy: 0.9793 - val_recall_4: 0.0000e+00
Epoch 35/1000
2050/2050 [==============================] - 12s 6ms/step - loss: 0.0595 - precision_4: 0.5328 - binary_accuracy: 0.9806 - recall_4: 0.0194 - val_loss: 0.2676 - val_precision_4: 0.0000e+00 - val_binary_accuracy: 0.9793 -

2050/2050 [==============================] - 12s 6ms/step - loss: 0.0517 - precision_4: 0.5917 - binary_accuracy: 0.9810 - recall_4: 0.0745 - val_loss: 0.3318 - val_precision_4: 0.0000e+00 - val_binary_accuracy: 0.9793 - val_recall_4: 0.0000e+00
Epoch 64/1000
2050/2050 [==============================] - 12s 6ms/step - loss: 0.0515 - precision_4: 0.6065 - binary_accuracy: 0.9811 - recall_4: 0.0798 - val_loss: 0.3351 - val_precision_4: 0.0000e+00 - val_binary_accuracy: 0.9793 - val_recall_4: 0.0000e+00
Epoch 65/1000
2050/2050 [==============================] - 12s 6ms/step - loss: 0.0512 - precision_4: 0.6045 - binary_accuracy: 0.9811 - recall_4: 0.0787 - val_loss: 0.3336 - val_precision_4: 0.0000e+00 - val_binary_accuracy: 0.9793 - val_recall_4: 0.0000e+00
Epoch 66/1000
2050/2050 [==============================] - 12s 6ms/step - loss: 0.0512 - precision_4: 0.6038 - binary_accuracy: 0.9811 - recall_4: 0.0787 - val_loss: 0.3382 - val_precision_4: 0.0000e+00 - val_binary_accuracy: 0.9793 -

...

2050/2050 [==============================] - 12s 6ms/step - loss: 0.0332 - precision_4: 0.7110 - binary_accuracy: 0.9858 - recall_4: 0.4529 - val_loss: 0.5988 - val_precision_4: 0.0219 - val_binary_accuracy: 0.9776 - val_recall_4: 0.0019
Epoch 864/1000
2050/2050 [==============================] - 12s 6ms/step - loss: 0.0336 - precision_4: 0.7048 - binary_accuracy: 0.9856 - recall_4: 0.4466 - val_loss: 0.5965 - val_precision_4: 0.0231 - val_binary_accuracy: 0.9778 - val_recall_4: 0.0018
Epoch 865/1000
2050/2050 [==============================] - 12s 6ms/step - loss: 0.0333 - precision_4: 0.7112 - binary_accuracy: 0.9858 - recall_4: 0.4504 - val_loss: 0.6006 - val_precision_4: 0.0264 - val_binary_accuracy: 0.9779 - val_recall_4: 0.0019
Epoch 866/1000
2050/2050 [==============================] - 12s 6ms/step - loss: 0.0334 - precision_4: 0.7071 - binary_accuracy: 0.9856 - recall_4: 0.4456 - val_loss: 0.6016 - val_precision_4: 0.0251 - val_binary_accuracy: 0.9776 - val_recall_4: 0.0022
Epo

2050/2050 [==============================] - 12s 6ms/step - loss: 0.0332 - precision_4: 0.7105 - binary_accuracy: 0.9858 - recall_4: 0.4506 - val_loss: 0.6051 - val_precision_4: 0.0289 - val_binary_accuracy: 0.9776 - val_recall_4: 0.0025
Epoch 896/1000
2050/2050 [==============================] - 12s 6ms/step - loss: 0.0332 - precision_4: 0.7075 - binary_accuracy: 0.9857 - recall_4: 0.4527 - val_loss: 0.6064 - val_precision_4: 0.0238 - val_binary_accuracy: 0.9778 - val_recall_4: 0.0019
Epoch 897/1000
2050/2050 [==============================] - 12s 6ms/step - loss: 0.0336 - precision_4: 0.7051 - binary_accuracy: 0.9856 - recall_4: 0.4486 - val_loss: 0.6042 - val_precision_4: 0.0269 - val_binary_accuracy: 0.9776 - val_recall_4: 0.0023
Epoch 898/1000
2050/2050 [==============================] - 12s 6ms/step - loss: 0.0331 - precision_4: 0.7111 - binary_accuracy: 0.9858 - recall_4: 0.4524 - val_loss: 0.6077 - val_precision_4: 0.0264 - val_binary_accuracy: 0.9775 - val_recall_4: 0.0025
Epo

2050/2050 [==============================] - 12s 6ms/step - loss: 0.0330 - precision_4: 0.7112 - binary_accuracy: 0.9858 - recall_4: 0.4544 - val_loss: 0.6134 - val_precision_4: 0.0211 - val_binary_accuracy: 0.9778 - val_recall_4: 0.0016
Epoch 928/1000
2050/2050 [==============================] - 12s 6ms/step - loss: 0.0331 - precision_4: 0.7088 - binary_accuracy: 0.9858 - recall_4: 0.4564 - val_loss: 0.6096 - val_precision_4: 0.0188 - val_binary_accuracy: 0.9778 - val_recall_4: 0.0014
Epoch 929/1000
2050/2050 [==============================] - 12s 6ms/step - loss: 0.0332 - precision_4: 0.7120 - binary_accuracy: 0.9858 - recall_4: 0.4534 - val_loss: 0.6129 - val_precision_4: 0.0247 - val_binary_accuracy: 0.9778 - val_recall_4: 0.0019
Epoch 930/1000
2050/2050 [==============================] - 12s 6ms/step - loss: 0.0333 - precision_4: 0.7098 - binary_accuracy: 0.9858 - recall_4: 0.4520 - val_loss: 0.6077 - val_precision_4: 0.0188 - val_binary_accuracy: 0.9779 - val_recall_4: 0.0013
Epo

2050/2050 [==============================] - 12s 6ms/step - loss: 0.0330 - precision_4: 0.7137 - binary_accuracy: 0.9859 - recall_4: 0.4560 - val_loss: 0.6090 - val_precision_4: 0.0208 - val_binary_accuracy: 0.9778 - val_recall_4: 0.0016
Epoch 960/1000
2050/2050 [==============================] - 12s 6ms/step - loss: 0.0331 - precision_4: 0.7133 - binary_accuracy: 0.9859 - recall_4: 0.4589 - val_loss: 0.6110 - val_precision_4: 0.0183 - val_binary_accuracy: 0.9778 - val_recall_4: 0.0014
Epoch 961/1000
2050/2050 [==============================] - 12s 6ms/step - loss: 0.0328 - precision_4: 0.7104 - binary_accuracy: 0.9858 - recall_4: 0.4570 - val_loss: 0.6109 - val_precision_4: 0.0233 - val_binary_accuracy: 0.9777 - val_recall_4: 0.0019
Epoch 962/1000
2050/2050 [==============================] - 12s 6ms/step - loss: 0.0331 - precision_4: 0.7123 - binary_accuracy: 0.9859 - recall_4: 0.4573 - val_loss: 0.6149 - val_precision_4: 0.0238 - val_binary_accuracy: 0.9779 - val_recall_4: 0.0018
Epo

2050/2050 [==============================] - 12s 6ms/step - loss: 0.0331 - precision_4: 0.7101 - binary_accuracy: 0.9858 - recall_4: 0.4555 - val_loss: 0.6106 - val_precision_4: 0.0316 - val_binary_accuracy: 0.9776 - val_recall_4: 0.0027
Epoch 992/1000
2050/2050 [==============================] - 12s 6ms/step - loss: 0.0330 - precision_4: 0.7086 - binary_accuracy: 0.9858 - recall_4: 0.4542 - val_loss: 0.6136 - val_precision_4: 0.0259 - val_binary_accuracy: 0.9780 - val_recall_4: 0.0018
Epoch 993/1000
2050/2050 [==============================] - 12s 6ms/step - loss: 0.0329 - precision_4: 0.7110 - binary_accuracy: 0.9859 - recall_4: 0.4588 - val_loss: 0.6098 - val_precision_4: 0.0316 - val_binary_accuracy: 0.9777 - val_recall_4: 0.0026
Epoch 994/1000
2050/2050 [==============================] - 12s 6ms/step - loss: 0.0330 - precision_4: 0.7105 - binary_accuracy: 0.9858 - recall_4: 0.4580 - val_loss: 0.6114 - val_precision_4: 0.0253 - val_binary_accuracy: 0.9778 - val_recall_4: 0.0020
Epo